# Road Traffic Vehicle Counting Exam

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

In [3]:
from PIL import Image
import os
import tensorflow as tf
import keras_cv
import tensorflow_datasets as tfds
import keras
import cv2
import numpy as np
from ultralytics import YOLO
from keras_cv import bounding_box
from keras_cv import visualization
import tqdm

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [2]:
!pip install ultralytics
!pip install keras_cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 56.4 MB/s eta 0:00:00


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [4]:
model = YOLO('/content/drive/MyDrive/Datasets/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [5]:
video_path = '/content/drive/MyDrive/Datasets/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [6]:
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [7]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (w, h))


## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [8]:
#region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360)]

roi_top = (100, 0, w-100, 100)
roi_bottom = (100, h-100, w-100, h)
roi_left = (0, 100, 100, h-100)
roi_right = (w-100, 100, w, h-100)

In [9]:
count_top = count_bottom = count_left = count_right = 0

In [10]:
def point_in_rect(point, rect):
    x, y = point
    x1, y1, x2, y2 = rect
    return x1 < x < x2 and y1 < y < y2

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [11]:
frame_count = 0
tracked_objects = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1


    results = model(frame)


    for r in results:
        boxes = r.boxes
        for box in boxes:

            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)


            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2


            conf = float(box.conf)
            cls = int(box.cls)


            obj_id = f"{center_x}_{center_y}"

            if obj_id not in tracked_objects:
                tracked_objects[obj_id] = {"last_pos": (center_x, center_y), "counted": False}


            last_pos = tracked_objects[obj_id]["last_pos"]
            if not tracked_objects[obj_id]["counted"]:
                if point_in_rect(last_pos, roi_top) and not point_in_rect((center_x, center_y), roi_top):
                    count_top += 1
                    tracked_objects[obj_id]["counted"] = True
                elif point_in_rect(last_pos, roi_bottom) and not point_in_rect((center_x, center_y), roi_bottom):
                    count_bottom += 1
                    tracked_objects[obj_id]["counted"] = True
                elif point_in_rect(last_pos, roi_left) and not point_in_rect((center_x, center_y), roi_left):
                    count_left += 1
                    tracked_objects[obj_id]["counted"] = True
                elif point_in_rect(last_pos, roi_right) and not point_in_rect((center_x, center_y), roi_right):
                    count_right += 1
                    tracked_objects[obj_id]["counted"] = True

            tracked_objects[obj_id]["last_pos"] = (center_x, center_y)


            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)


            cv2.putText(frame, f"Conf: {conf:.2f}, ID: {obj_id}, Class: {cls}",
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)


    cv2.rectangle(frame, (roi_top[0], roi_top[1]), (roi_top[2], roi_top[3]), (255, 0, 0), 2)
    cv2.rectangle(frame, (roi_bottom[0], roi_bottom[1]), (roi_bottom[2], roi_bottom[3]), (255, 0, 0), 2)
    cv2.rectangle(frame, (roi_left[0], roi_left[1]), (roi_left[2], roi_left[3]), (255, 0, 0), 2)
    cv2.rectangle(frame, (roi_right[0], roi_right[1]), (roi_right[2], roi_right[3]), (255, 0, 0), 2)


    cv2.putText(frame, f"top: {count_top}", (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"bottom: {count_bottom}", (10, 70), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"left: {count_left}", (10, 110), cv2.FONT_HERSHEY_COMPLEX , 1, (0, 0, 255), 2)
    cv2.putText(frame, f"right: {count_right}", (10, 150), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, "Naif abdullah alsabhan", (w - 200, 30), cv2. FONT_ITALIC , 1, (255, 255, 255), 2)


    out.write(frame)
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"total  vehicles counted: top: {count_top}, counted  bottom: {count_bottom}, counted left: {count_left}, counted right: {count_right}")


0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 109.9ms
Speed: 13.1ms preprocess, 109.9ms inference, 824.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 53.6ms
Speed: 3.9ms preprocess, 53.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 53.5ms
Speed: 3.2ms preprocess, 53.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 53.5ms
Speed: 3.1ms preprocess, 53.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 47.0ms
Speed: 3.3ms preprocess, 47.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 47.0ms
Speed: 3.7ms preprocess, 47.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 40.9ms
Speed: 6.7ms preprocess, 40.9ms inf

## Save and Submit Your Work

In [ ]:
#cap.release()
#out.release()
#cv2.destroyAllWindows()

Submit both the notebook and the output video